In [3]:
import pandas as pd
import numpy as np
import json

csv_file_path = "logit_llama70b.csv"
df = pd.read_csv(csv_file_path)

def softmax(logits):
    exp_logits = np.exp(logits - np.max(logits)) 
    return exp_logits / np.sum(exp_logits)

def entropy(probabilities):
    return -np.sum(probabilities * np.log(probabilities + 1e-9))  

def calculate_entropy_from_logits(logits):
    logits = np.array(logits)
    probabilities = softmax(logits)
    return entropy(probabilities)

mmlu_entropies = []
dgen_entropies = []

for _, row in df.iterrows():
    mmlu_logits = [row["mmlu_correct_logit"]] + json.loads(row["mmlu_distractor_logits"])
    dgen_logits = [row["dgen_correct_logit"]] + json.loads(row["dgen_distractor_logits"])

    # entropy calculation
    mmlu_entropy = calculate_entropy_from_logits(mmlu_logits)
    dgen_entropy = calculate_entropy_from_logits(dgen_logits)

    mmlu_entropies.append(mmlu_entropy)
    dgen_entropies.append(dgen_entropy)

df["mmlu_entropy"] = mmlu_entropies
df["dgen_entropy"] = dgen_entropies

subject_entropy_means = df.groupby("subject")[["mmlu_entropy", "dgen_entropy"]].mean()

output_csv_path = "logit_llama70_entropy.csv"
subject_entropy_means.to_csv(output_csv_path)

subject_entropy_means


,mmlu_entropy,dgen_entropy
subject,,
abstract_algebra,0.879201,0.981834
anatomy,0.776535,0.703233
astronomy,0.832648,0.847327
business_ethics,0.903566,0.816202
clinical_knowledge,0.758999,0.834860
college_biology,0.815545,0.862225
college_chemistry,0.859810,0.872216
college_computer_science,0.957541,0.926957
college_mathematics,0.893979,0.941948


In [5]:
## domain mapping


import pandas as pd

files = [
    # "logit_qwen_entropy.csv",
    "logit_llama70_entropy.csv",
    # "logit_mixtral_entropy.csv"
]

domain_mapping = {
    "Humanities": [
        "Formal Logic", "High School European History", "High School US History", 
        "High School World History", "International Law", "Jurisprudence", 
        "Logical Fallacies", "Moral Disputes", "Moral Scenarios", "Philosophy", 
        "Prehistory", "Professional Law", "World Religions"
    ],
    "Social Sciences": [
        "Econometrics", "High School Geography", "High School Government and Politics", 
        "High School Macroeconomics", "High School Microeconomics", "High School Psychology", 
        "Professional Psychology", "Public Relations", "Security Studies", "Sociology", 
        "US Foreign Policy", "Human Sexuality"
    ],
    "STEM": [
        "Abstract Algebra", "Anatomy", "Astronomy", "College Biology", "College Chemistry", 
        "College Computer Science", "College Mathematics", "College Physics", "Computer Security", 
        "Conceptual Physics", "Electrical Engineering", "Elementary Mathematics", 
        "High School Biology", "High School Chemistry", "High School Computer Science", 
        "High School Mathematics", "High School Physics", "High School Statistics", "Machine Learning"
    ],
    "Others": [
        "Business Ethics", "Clinical Knowledge", "College Medicine", "Global Facts", 
        "Human Aging", "Management", "Marketing", "Medical Genetics", "Miscellaneous", 
        "Nutrition", "Professional Accounting", "Professional Medicine", "Virology"
    ]
}

reverse_mapping = {subject: domain for domain, subjects in domain_mapping.items() for subject in subjects}

for file in files:
    df = pd.read_csv(file)

    # Add 'domain' column based on mapping
    df["domain"] = df["subject"].map(lambda x: reverse_mapping.get(x.replace("_", " ").title(), "Unknown"))

    df.sort_values(by=["domain", "subject"], inplace=True)

    df.to_csv(file, index=False)

print("Domain mapping and sorting complete. Updated files saved.")


Domain mapping and sorting complete. Updated files saved.


In [9]:
import pandas as pd
import scipy.stats as stats

files = [
    # "logit_qwen_entropy.csv",
    "logit_llama70_entropy.csv",
    # "logit_mixtral_entropy.csv"
]

for file in files:
    df = pd.read_csv(file)

    unique_domains = df["domain"].unique()
    for domain in unique_domains:
        domain_df = df[df["domain"] == domain].copy() 
        domain_df.loc[:, "entropy_diff"] = domain_df["mmlu_entropy"] - domain_df["dgen_entropy"]

        print(f"{file} - {domain}: Wilcoxon signed-rank test")

        try:
            stat, p_value = stats.wilcoxon(domain_df["mmlu_entropy"], domain_df["dgen_entropy"])
            print(f"Wilcoxon signed-rank test Statistic: {stat:.4f}, p-value: {p_value:.4f}")

            if p_value < 0.05:
                print(f"Significant difference in {domain} (p < 0.05)\n")
            else:
                print(f"No Significant difference in {domain} (p ≥ 0.05)\n")

        except ValueError:
            print(f"Error in {domain}")


logit_llama70_entropy.csv - Humanities: Wilcoxon signed-rank test
Wilcoxon signed-rank test Statistic: 13.0000, p-value: 0.0425
Significant difference in Humanities (p < 0.05)

logit_llama70_entropy.csv - Others: Wilcoxon signed-rank test
Wilcoxon signed-rank test Statistic: 29.0000, p-value: 0.2734
No Significant difference in Others (p ≥ 0.05)

logit_llama70_entropy.csv - STEM: Wilcoxon signed-rank test
Wilcoxon signed-rank test Statistic: 68.0000, p-value: 0.2935
No Significant difference in STEM (p ≥ 0.05)

logit_llama70_entropy.csv - Social Sciences: Wilcoxon signed-rank test
Wilcoxon signed-rank test Statistic: 12.0000, p-value: 0.1309
No Significant difference in Social Sciences (p ≥ 0.05)

logit_llama70_entropy.csv - Unknown: Wilcoxon signed-rank test
Wilcoxon signed-rank test Statistic: 1.0000, p-value: 0.5000
No Significant difference in Unknown (p ≥ 0.05)

